# Extracting MIMIC Data

## Environment Setup

In [ ]:
import time
import pymysql
import getpass
import pickle as pkl
from _collections import OrderedDict

## Query List

#### Construct a list of queries that we will want to run.

In [ ]:
queryList = OrderedDict([('PNA', 'PNA-Mimic.sql'), ('Trauma','Trauma-Mimic.sql')])

## MIMIC Database Connection

#### Make a connection to the MIMIC database and get a cursor for record processing.

In [ ]:
conn = pymysql.connect(host="mysql", 
                       port = 3306, user="jovyan", 
                       passwd=getpass.getpass("Enter MySQL passwd for jovyan"), db='mimic2')
cur = conn.cursor()

## Retrieve our Data

#### For each query we will retrieve the data and build a ordered dictionary containing our data. Were use an ordered dictionary because we want to keep the Pneumonia and Trauma cases together.

In [ ]:
corpus = OrderedDict()
for key in queryList:
    count = 1
    file = open(queryList[key], 'r')
    query = file.read()
    print("execute query: " + key)
    %time cur.execute(query)
    for row in cur:
        ckey = key + str(count)
        corpus[ckey] = row[0]
        count += 1
    file.close()

## Safestore the Data

#### We will serialize the ordered dictionary structure out to disk. This waty we will not have to rebuild this data structure when we want to use our data. It is a convenience thing....

In [ ]:
file = open('classification-corpus.pkl','wb')
pkl.dump(corpus, file)
file.close()
conn.close()
print("Done!")